# Project (Section 10)

This is where we left off in the previous section project:

In [1]:
countries = {
    "australia": ("Australia", "AUS"),
    "canada": ("Canada", "CAN"),
    "china": ("China", "CHN"),
    "france": ("France", "FRA"),
    "germany": ("Germany", "DEU"),
    "india": ("India", "IND"),
    "mexico": ("Mexico", "MEX"),
    "norway": ("Norway", "NOR"),
    "pakistan": ("Pakistan", "PAK"),
    "san marino": ("San Marino", "SMR"),
    "sanmarino": ("San Marino", "SMR"),
    "spain": ("Spain", "ESP"),
    "sweden": ("Sweden", "SWE"),
    "united kingdom": ("United Kingdom", "GBR"),
    "uk": ("United Kingdom", "GBR"),
    "great britain": ("United Kingdom", "GBR"),
    "britain": ("United Kingdom", "GBR"),
    "us": ("United States of America", "USA"),
    "united states": ("United States of America", "USA"),
    "usa": ("United States of America", "USA"),
}
valid_country_names = sorted(countries.keys())
valid_country_names

def lookup_country(name: str) -> tuple[str, str]:
    name = name.strip().casefold()
    
    try:
        return countries[name]
    except KeyError:
        raise ValueError(
            "Unknown country name. "
            f"Country name must be one of: {','.join(valid_country_names)}"
        )

country_code_lookup = {
    name: code
    for name, code in countries.values()
}

In [2]:
from datetime import date
from enum import Enum
from functools import cached_property
from typing import Annotated, TypeVar
from uuid import uuid4
from pydantic import (
    AfterValidator, 
    BaseModel, 
    ConfigDict, 
    computed_field,
    Field, 
    field_serializer, 
    field_validator,
    UUID4,
    ValidationInfo,
)
from pydantic.alias_generators import to_camel


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


T = TypeVar('T')
BoundedString = Annotated[str, Field(min_length=2, max_length=50)]
BoundedList = Annotated[list[T], Field(min_length=1, max_length=5)]


Country = Annotated[str, AfterValidator(lambda name: lookup_country(name)[0])]


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = Field(default=False, repr=False)
    manufactured_date: date = Field(
        validation_alias="completionDate", 
        ge=date(1980, 1, 1), 
        repr=False
    )
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD",
        repr=False,
    )
    top_features: BoundedList[BoundedString] | None = Field(default=None, repr=False)
    vin: BoundedString = Field(repr=False)
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
        repr=False,
    )
    registration_country: Country | None = Field(default=None, repr=False)
    
    @computed_field(repr=False)
    @cached_property
    def registration_country_code(self) -> str:
        return country_code_lookup[self.registration_country] 
    
    registration_date: date | None = Field(default=None, repr=False)
    license_plate: BoundedString | None = Field(default=None, repr=False)

    
    @field_serializer("manufactured_date", "registration_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")
        
    @field_validator("registration_date")
    @classmethod
    def validate_registration_date(cls, value:date, values: ValidationInfo):
        data = values.data
        if "manufactured_date" in data and data["manufactured_date"] > value:
            raise ValueError("Automobile cannot be registered prior to manufacture date.")
        return value

Since our date serializer is being used more than once, an Annotated type might be a better approach - we can get that logic out of the model, and simplify the way our model looks a little bit.

So, for this exercise, make an annotated type for the date serialization, and use that annotated type in your model. Name the annotated type `CustomDate`.

You can re-use some previous sample data to test your model.